In [119]:
from sqlalchemy import create_engine
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from env_vars import *
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import pickle
import numpy as np

# Connection to DB for executions

In [2]:
engine = create_engine('sqlite:///song_list_v2.db', echo=False)

# Functions for use in this notebook

In [3]:
def authorize():
    client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
def get_features(trackID):
        return sp.audio_features(trackID)[0]
    
get_features('54flyrjcdnQdco7300avMJ')

{'danceability': 0.693,
 'energy': 0.497,
 'key': 2,
 'loudness': -7.316,
 'mode': 1,
 'speechiness': 0.119,
 'acousticness': 0.679,
 'instrumentalness': 0,
 'liveness': 0.258,
 'valence': 0.473,
 'tempo': 81.308,
 'type': 'audio_features',
 'id': '54flyrjcdnQdco7300avMJ',
 'uri': 'spotify:track:54flyrjcdnQdco7300avMJ',
 'track_href': 'https://api.spotify.com/v1/tracks/54flyrjcdnQdco7300avMJ',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/54flyrjcdnQdco7300avMJ',
 'duration_ms': 122067,
 'time_signature': 4}

In [6]:
def get_artist_id(trackID):
    return sp.track(trackID)['artists'][0]['id']
    


get_artist_id('54flyrjcdnQdco7300avMJ')

'1dfeR4HaWDbWqFHLkxsg1d'

In [7]:
def get_genres(artistID):
    return sp.artist(artistID)['genres']


get_genres('1dfeR4HaWDbWqFHLkxsg1d')

['glam rock', 'rock']

# [COMPLETED] Inserts Genre Value into Genres Column

In [ ]:
for item in engine.execute("SELECT songid FROM features").fetchall()[515122:]:    
    songid = item.values()[0]
    artistid = get_artist_id(songid)
    genres = get_genres(artistid)
    try:
        engine.execute(f'UPDATE features SET genres = \"{genres}\" WHERE songid = \"{songid}\"')
        authorize()
    except:
        print("couldn't insert")
        continue

In [13]:
# null = []
# for genre in genres_list:
#     if genre == []:
#         null.append(genre)

# Convert to PD DF - For Exploration - SEE NULL VALUE RESULT

In [103]:
con = sqlite3.connect("./song_list_v2.db")
df = pd.read_sql_query("SELECT * from features", con)

In [65]:
df.isnull().sum()

index               0
acousticness        0
artist              0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
songid              0
speechiness         0
tempo               0
time_signature      0
track               1
valence             0
genres              0
dtype: int64

# [Complete] - Create List of all Null Values for Investigation

In [69]:
nan_rows = df[df['genres'].isnull()].index.tolist()

In [70]:
len(nan_rows)

0

In [71]:
# df.loc[nan_rows[0]]

In [72]:
df.loc[2:5]

,index,acousticness,artist,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,songid,speechiness,tempo,time_signature,track,valence,genres
2,2,0.000725,Der Mystic,0.430,514290.0,0.960,0.925000,5.0,0.123,-7.741,0.0,2poHURuOfVNbzZdivAwtOH,0.0431,140.001,4.0,Tangle Of Aspens,0.225,[]
3,3,0.245000,Sasha / Turbulence,0.663,212413.0,0.677,0.000000,3.0,0.350,-4.117,0.0,1jg9hZnReygpBvV2axGuPy,0.0755,152.085,4.0,We Have Got Love,0.879,[]
4,4,0.002760,Kris Kross,0.859,221200.0,0.741,0.000000,11.0,0.325,-12.329,0.0,3GsS8jzoixpCnp4jDWCEvb,0.2710,98.082,4.0,2 Da Beat Ch'yall,0.529,"['hip hop', 'new jack swing', 'rap']"
5,5,0.348000,Jorge Negrete,0.361,173573.0,0.483,0.000003,7.0,0.177,-6.875,1.0,0kq75szR7uDEYrZkT2c4Ry,0.0287,94.538,4.0,El hijo del pueblo,0.682,"['bolero', 'mariachi', 'ranchera']"


# Remove track NULL value and reset index and pickle new DF

In [104]:
df = df.reset_index()

In [105]:
df = df.drop(columns="index")

In [106]:
df.loc[[117524], ['track']] = "Mashin For Mine / Summer In The City"

In [107]:
df.genres.value_counts()

[]                                                                                                                                                                                                        154138
['baroque', 'classical', 'early music', 'german baroque']                                                                                                                                                   3654
['classical', 'classical era']                                                                                                                                                                              1907
['cartoon','children's music']                                                                                                                                                                              1468
['smooth jazz']                                                                                                                                                     

In [108]:
indexNames = df[df['genres'] == "[]" ].index

In [109]:
df.drop(indexNames , inplace=True)

In [110]:
df.describe()

,level_0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,574018.000000,574018.000000,574018.000000,5.740180e+05,574018.000000,574018.000000,574018.000000,574018.000000,574018.000000,574018.000000,574018.000000,574018.000000,574018.000000,574018.000000
mean,357946.562946,0.317023,0.538694,2.408866e+05,0.612956,0.208388,5.284643,0.214476,-9.581242,0.662383,0.089784,120.748153,3.879729,0.493747
std,208887.321521,0.341293,0.183636,1.195066e+05,0.261756,0.337502,3.562836,0.191707,5.209874,0.472897,0.113566,30.142483,0.478231,0.267250
min,0.000000,0.000000,0.000000,1.161000e+03,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,177522.250000,0.011900,0.411000,1.789322e+05,0.422000,0.000001,2.000000,0.096400,-11.955000,0.000000,0.035400,97.679000,4.000000,0.272000
50%,355701.500000,0.161000,0.550000,2.235070e+05,0.649000,0.000905,5.000000,0.133000,-8.399000,1.000000,0.048300,120.005000,4.000000,0.493000
75%,535066.750000,0.613000,0.677000,2.782760e+05,0.838000,0.341000,9.000000,0.278000,-5.932000,1.000000,0.086800,139.680000,4.000000,0.714000
max,728154.000000,0.996000,0.997000,4.830606e+06,1.000000,1.000000,11.000000,1.000000,6.336000,1.000000,0.967000,249.859000,5.000000,1.000000


In [115]:
pickle.load(df, open( "./final_data/DF_v3.pkl", "wb" ) )

In [127]:
# unique_genres = np.unique(df[["genres"]])

In [160]:
all_genres = []
for genre_list in df["genres"]:
    genre_list = genre_list.split(",")
    for genre in genre_list:
        genre = genre.strip("[",)
        genre = genre.strip("]",)
        genre = genre.strip()
        all_genres.append(genre)

In [161]:
len(all_genres)

2110517

In [162]:
all_genres[0:4]

["'finnish indie'", "'suomi rock'", "'bass music'", "'scottish electronic'"]

In [168]:
df_genres = pd.DataFrame(all_genres,columns=['genres'])

In [197]:
len(df_genres)

2110517

In [212]:
3488 - 2906

582

In [218]:
(df['genres'].value_counts() > 100).sum()

848

In [195]:
df_genres[df_genres[condition] == True]

/Users/user/.local/share/virtualenvs/data_collection-PY2mH6bz/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [176]:
unique_genres = set(df_genres["genres"])

In [177]:
len(unique_genres)

3488

# Print Values for Test - Complete

In [116]:
def print_values(songid):    
    artistid = get_artist_id(songid)
    print(artistid)
    genres = get_genres(artistid)
    print(genres)

print_values('4LisLrspORq8IagdLHWkf7')

4zMssRZ6iG7sJm54RwFozt
['cartoon', "children's music"]


# [Completed] 
# Retry Genre Lookup / SQL Update on Collected Db for Null Rows

In [75]:
nan_rows

[]

In [77]:
# for row_index in nan_rows:
#     songid = df.loc[row_index]["songid"]
#     artistid = get_artist_id(songid)
#     genres = get_genres(artistid)
#     newgenres = '['
#     for item in g:
#         newgenres += "'" + item + "',"
#     newgenres = words[:-1]
#     newgenres += "]" 
#     try:
#         engine.execute(f'UPDATE features SET genres = \"{newgenres}\" WHERE songid = \"{songid}\"')
#         authorize()
#     except:
#         print(f"couldn't insert {songid} into table")
#         continue
        